In [2]:
import numpy as np
from PIL import Image

def mod_inverse(a, m):
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return None

def encrypt_block(block, key_matrix):
    return np.dot(key_matrix, block) % 256

def decrypt_block(block, inv_key_matrix):
    return np.dot(inv_key_matrix, block) % 256

def compute_inverse_key_matrix(key_matrix, mod=256):
    det = int(np.round(np.linalg.det(key_matrix)))
    det_inv = mod_inverse(det, mod)
    if det_inv is None:
        raise ValueError("Key matrix is not invertible under mod 256")
    adjugate_matrix = np.round(det * np.linalg.inv(key_matrix)).astype(int) % mod
    return (det_inv * adjugate_matrix) % mod

def hill_cipher_image(image_path, key_matrix, mode='encrypt'):

    img = Image.open(image_path).convert('L')
    pixels = np.array(img)
    n = key_matrix.shape[0]

    inv_key_matrix = None
    if mode == 'decrypt':
        inv_key_matrix = compute_inverse_key_matrix(key_matrix)

    padded_height = (pixels.shape[0] + n - 1) // n * n
    padded_width = (pixels.shape[1] + n - 1) // n * n
    padded_pixels = np.pad(pixels, ((0, padded_height - pixels.shape[0]),
                                    (0, padded_width - pixels.shape[1])),
                           mode='constant', constant_values=0)
    processed_pixels = np.copy(padded_pixels)

    for i in range(0, padded_pixels.shape[0], n):
        for j in range(0, padded_pixels.shape[1], n):
            block = padded_pixels[i:i+n, j:j+n]
            if mode == 'encrypt':
                processed_block = encrypt_block(block, key_matrix)
            else:
                processed_block = decrypt_block(block, inv_key_matrix)
            processed_pixels[i:i+n, j:j+n] = processed_block

    final_pixels = processed_pixels[:pixels.shape[0], :pixels.shape[1]]
    processed_img = Image.fromarray(final_pixels.astype(np.uint8))
    output_path = f"{image_path.split('.')[0]}_{mode}.png"
    processed_img.save(output_path)

    return output_path

key_matrix = np.array([
    [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [6, 8, 10, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 8, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
])


image_path = 'images.jpeg'

encrypted_image_path = hill_cipher_image(image_path, key_matrix, mode='encrypt')
print(f"Encrypted image saved to: {encrypted_image_path}")

decrypted_image_path = hill_cipher_image(encrypted_image_path, key_matrix, mode='decrypt')
print(f"Decrypted image saved to: {decrypted_image_path}")


Encrypted image saved to: images_encrypt.png
Decrypted image saved to: images_encrypt_decrypt.png
